In [ ]:
import pandas as pd
from padagua_2021.Geolocalizacion.separacion_domicilios.funciones_diccionarios import *

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from padagua_2021.Geolocalizacion.separacion_domicilios.funciones_diccionarios import diccionarios_gcp  


In [ ]:
E_P=pd.read_excel('Downloads/Ecatepec de Morelos_PROPIETARIO (2).xlsx')
E_PC=pd.read_excel('Downloads/Ecatepec de Morelos_PREDIAL (1).xlsx')
E_PP=pd.read_csv('Downloads/Ectepec_de_Morelos_PREDIALADEUDO_Consolidado.csv', encoding='ANSI', sep='\t')

In [ ]:
E_PC=E_PC.loc[~E_PC['CLAVECATASTRAL'].str.contains('PRUEB')]
# E_PC.loc[(~E_PC['DIRECCION'].str.contains('SIN NOMBRE|S/N|S-N|#¿NOMBRE?|MZ|0 . .')) &(E_PC['DIRECCION']!='')].DIRECCION.value_counts()

In [ ]:
E_PC.loc[E_PC['DIRECCION'].str.contains('MZ')].DIRECCION.value_counts()

# Código de prueba

In [ ]:
CAT=pd.read_excel('Downloads/claves_catastrales (1).xlsx')


In [ ]:
CAT

In [ ]:
CAT2=pd.read_excel('Downloads/claves_catastrales.xlsx')

In [ ]:
PRED_CAT=pd.merge(CAT, E_PC, left_on='Clave Catastral', right_on='CLAVECATASTRAL')
PRED_CAT['CLAVE_PRED']= PRED_CAT['CLAVECATASTRAL'].str[:10]
E_PC['CLAVE_PRED']= E_PC['CLAVECATASTRAL'].str[:10]
CAT['CLAVE_PRED']= CAT['Clave Catastral'].str[:10]
NOT_IN_C= E_PC.loc[~E_PC['CLAVE_PRED'].isin(PRED_CAT['CLAVE_PRED'])]
NOT_IN_P=CAT.loc[~CAT['Clave Catastral'].isin(PRED_CAT['CLAVECATASTRAL'])]

In [ ]:
NOT_IN_C=NOT_IN_C.loc[NOT_IN_C['CLAVEZONA']!=11]
NOT_IN_P
PRED_CAT=PRED_CAT.loc[PRED_CAT['CLAVEZONA']!=11].sort_values('Número de Unidad', ascending=False).drop_duplicates('CLAVECATASTRAL',keep='first').drop(columns=['CLAVE_PRED_x','CLAVE_PRED_y'])

In [ ]:
PRED_CAT.to_csv('CRUCE_PREDIAL_CATASTRO.csv', encoding='utf-8-sig')
NOT_IN_C.to_csv('PREDIAL_FUERA_CATASTRO.csv', encoding='utf-8-sig')
NOT_IN_P.to_csv('CATASTRO_FUERA_PREDIAL.csv', encoding='utf-8-sig')

In [ ]:
CAT.loc[~CAT['Clave Catastral'].isin(CAT2['Clave Catastral'])]

In [ ]:
CAT2.loc[~CAT2['Clave Catastral'].isin(CAT['Clave Catastral'])]


In [ ]:
CAT.loc[(CAT['Domicilio del Inmueble'].astype(str) != CAT2['Domicilio del Inmueble'].astype(str))]


In [ ]:
CAT.loc[(CAT['Número de Unidad'].astype(str) != CAT2['Número de Unidad'].astype(str))]

In [ ]:
CAT.loc[(~CAT['Domicilio del Inmueble'].astype(str).str.contains('SIN NOMBRE|-|S/N')) & (CAT['Domicilio del Inmueble']!='0')& (CAT['Domicilio del Inmueble']!='.')]['Domicilio del Inmueble'].value_counts()

In [ ]:
CAT.rename(columns={'Domicilio del Inmueble':'BDINTERNA_CONT_DIRECCION'}, inplace=True)

In [ ]:
base_folder= './'

In [ ]:
CAT.drop_duplicates('Clave Catastral', inplace=True)

In [ ]:
CAT=procesamiento_con_diccionarios(CAT, base_folder, "BDINTERNA_CONT_")

In [ ]:
LISTA= pd.merge(CAT, E_PC, left_on='Clave Catastral', right_on='CLAVECATASTRAL')[['CLAVECATASTRAL','BDINTERNA_CONT_DIRECCION', 'DIRECCION']]

In [ ]:
LISTA.rename(columns={'BDINTERNA_CONT_DIRECCION':'CAT_DIRECCION'}, inplace=True)
LISTA.rename(columns={'DIRECCION':'BDINTERNA_CONT_DIRECCION'}, inplace=True)

In [ ]:
LISTA=procesamiento_con_diccionarios(LISTA, base_folder, "BDINTERNA_CONT_")

In [ ]:
L=LISTA.loc[(~LISTA['BDINTERNA_CONT_DIRECCION'].astype(str).str.contains('SIN NOMBRE|-|S/N')) & (LISTA['BDINTERNA_CONT_DIRECCION']!='0')& (LISTA['BDINTERNA_CONT_DIRECCION']!='.') &(LISTA['BDINTERNA_CONT_N_EXT'].notna())]

In [ ]:
# L['BDINTERNA_CONT_DIRECCION']=L['BDINTERNA_CONT_DIRECCION'].str.strip().str.replace(r'\s+', ' ', regex=True).str.replace('.', '', regex=False).str.replace('#', '', regex=False)

L[['BDINTERNA_CONT_DIRECCION']].to_csv('ECATEPEC_CAT_QUERY.csv', encoding='utf-8-sig')

In [ ]:
LL=pd.read_csv('ECATEPEC_CAT_QUERY.csv', encoding='utf-8-sig')

In [ ]:

def nomina_limpieza(base_nom):
    base_folder= './'
    # base_nom = bases_externas_oracle()
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    base_nom['ORDEN'] = base_nom.reset_index(drop=True).index+1

    base_nom['DIRECCION'] = base_nom['DIRECCION'].str.replace('DIRECCION', '')
    base_nom['NUM_INT'] = base_nom["DIRECCION"].str[5:].str.findall(r"(\d{1}\-?\d{0,4})").str[1]

    BN = base_nom.loc[~((base_nom['DIRECCION'].astype(str) == 'X') | (base_nom['DIRECCION'] == '-') | (base_nom['DIRECCION'].str.contains('SIN NOMBRE|SIN NUMERO')) | (
        base_nom['DIRECCION'] == 'SN') | (base_nom['DIRECCION'] == 'S/N') | (base_nom['DIRECCION'] == '0') | (base_nom['DIRECCION'] == '1') | (base_nom['NUM_EXT'].str.contains('SN')))]

    BN['CONST_DOM'] = BN['DIRECCION'].astype(str).str.cat(BN[['NUM_EXT', 'NUM_INT', 'COLONIA']].astype(str), ', ').str.replace(
        'None', '').str.replace(', ,', ', ').str.replace('  ', ' ').str.rstrip().str.rstrip(',').str.replace('.0', '', regex=False)
    BN.drop_duplicates('CONST_DOM', inplace=True)

    BN['PRE_DOM'] = BN['DIRECCION'].astype(str).str.cat(
        BN['NUM_EXT'].astype(str), ', ')
    BN['PRE_DOM'] = BN['DIRECCION'].astype(str)

    BN['N_EXT_2'] = BN.PRE_DOM.str.split(' MZ', expand=True)[1]
    BN['N_EXT_3'] = BN.PRE_DOM.str.split(' LT ', expand=True)[1]
    try:
        BN['N_EXT_4'] = BN.PRE_DOM.str.split(' PISO ', expand=True)[1]
    except:
        pass

    BN["PRE_DOM"] = BN["PRE_DOM"].str.split(' LT ', expand=True)[0]
    BN["PRE_DOM"] = BN["PRE_DOM"].str.split(' MZ ', expand=True)[0]
    BN["PRE_DOM"] = BN["PRE_DOM"].str.split(' LT.', expand=True)[0]
    BN["PRE_DOM"] = BN["PRE_DOM"].astype(str)
    try:
        BN["PRE_DOM"] = BN.apply(lambda row: row["PRE_DOM"].replace(
            str(row["N_EXT_4"]), ""), axis=1,)
    except:
        pass
    p = (BN["PRE_DOM"].str[5:].str.findall(r"(\d{1}\-?\d{0,4})").str.len())
    BN["len"] = p
    BN.loc[BN["len"] == 0, "N_EXT"] = 0
    BN["PRE_N_EXT"] = (BN["PRE_DOM"].str[5:].str.findall(r"(\d{1}\-?\d{0,4})"))
    BN.loc[BN["len"] == 1, "N_EXT"] = BN["PRE_N_EXT"].str[-1]
    BN.loc[BN["len"] == 2, "N_EXT"] = BN["PRE_N_EXT"].str[1]
    BN.loc[BN["len"] == 3, "N_EXT"] = BN["PRE_N_EXT"].str[0]

    BN["N_EXT"] = BN["N_EXT"].astype(str)
    BN["INTFIS_RS_FD_CALLE"] = BN.apply(
        lambda row: row["PRE_DOM"].replace(str(row["N_EXT_3"]), ""), axis=1,)
    BN["INTFIS_RS_FD_CALLE"] = BN.apply(
        lambda row: row["INTFIS_RS_FD_CALLE"].replace(str(row["N_EXT_2"]), ""), axis=1,)
    BN["INTFIS_RS_FD_CALLE"] = BN.apply(
        lambda row: row["INTFIS_RS_FD_CALLE"].replace(str(row["N_EXT"]), ""), axis=1,)

    BN["INTFIS_RS_FD_CALLE"] = BN["INTFIS_RS_FD_CALLE"].str.split(
        ' LT ', expand=True)[0]
    BN["INTFIS_RS_FD_CALLE"] = BN["INTFIS_RS_FD_CALLE"].str.split(
        ' MZ ', expand=True)[0]
    BN["INTFIS_RS_FD_CALLE"] = BN["INTFIS_RS_FD_CALLE"].str.split(
        ' LT.', expand=True)[0]
    BN["INTFIS_RS_FD_CALLE"] = BN["INTFIS_RS_FD_CALLE"].str.split(
        ' EDIF', expand=True)[0]
    BN["INTFIS_RS_FD_CALLE"] = BN["INTFIS_RS_FD_CALLE"].str.split(
        ' ESQ.', expand=True)[0]

    BN['INTFIS_RS_FD_CALLE'] = BN['INTFIS_RS_FD_CALLE'].astype(str).str.strip()

    BN.INTFIS_RS_FD_CALLE = BN.INTFIS_RS_FD_CALLE.str.replace(
        '&', 'Ñ', regex=False)
    BN.INTFIS_RS_FD_CALLE = BN.INTFIS_RS_FD_CALLE.str.replace(
        '¿', 'Ñ', regex=False)

    BN.loc[BN['NUM_EXT'] == '-', 'NUM_EXT'] = float('NaN')

    BN['INTFIS_RS_FD_N_EXT'] = BN['NUM_EXT'].combine_first(BN['N_EXT'])

    BN['DOM_UNICOS'] = BN['INTFIS_RS_FD_CALLE'].str.cat(
        BN[['INTFIS_RS_FD_N_EXT', 'COLONIA']], ' ').str.replace('.0', '', regex=False)
    BN['INTFIS_RS_FD_PRE_DOM'] = BN['INTFIS_RS_FD_CALLE'].str.cat(
        BN['INTFIS_RS_FD_N_EXT'], ' ').str.title()

    TBN = BN
    TBN['INTFIS_RS_FD_COLONIA'] = TBN['COLONIA'].str.replace(
        '.0', '', regex=False).str.title()
    TBN['INTFIS_RS_FD_DIRECCION'] = TBN['INTFIS_RS_FD_CALLE'].str.cat(
        TBN[['INTFIS_RS_FD_N_EXT']], ' ').str.cat(TBN['COLONIA'], ', ').str.replace('.0', '', regex=False)
    TBN.rename(columns={'CALLE': 'CALLE_NOMINA',
               'COLONIA': 'COLONIA_NOMINA'}, inplace=True)
    TBN = diccionarios_gcp(TBN)

    TBN['INTFIS_RS_FD_FULL_DOM'] = TBN['INTFIS_RS_FD_FULL_DOM'].str.strip()

    return(TBN)


In [ ]:
LISTA.rename(columns={'BDINTERNA_CONT_DIRECCION':'DIRECCION', 'BDINTERNA_CONT_N_EXT':'NUM_EXT'}, inplace=True)

In [ ]:
LISTA= nomina_limpieza(LISTA)

In [ ]:
pred= pd.read_csv('Downloads/prediosEcatepec_shapes.csv')

In [ ]:
pred['id_cat'] = pred['id_cat'].astype(str).str.zfill(16)
# CAT['Clave Catastral'] = CAT['Clave Catastral'].astype(str)

In [ ]:
# CAT['CLAVE_PRED']= CAT['Clave Catastral'].str[:10]
# E_PC['CLAVE_PRED']= E_PC['CLAVECATASTRAL'].str[:10]
pred['CLAVE_PRED']= pred['id_cat'].str[:10]

In [ ]:
LISTA['CLAVE_PRED']= LISTA['CLAVECATASTRAL'].str[:10]

In [ ]:
pred['COORDENADAS_RECAUDACION'] = pred['Latitude_clean'].astype(str).str.cat(pred['Longitude_clean'].astype(str), ', ')

In [ ]:
LISTA= pd.merge(CAT, E_PC, left_on='Clave Catastral', right_on='CLAVECATASTRAL', how='outer')

In [ ]:
LISTA.columns
LISTA['CLAVECATASTRAL']=LISTA['Clave Catastral'].combine_first(LISTA['CLAVECATASTRAL'])

In [ ]:
LISTA['CLAVE_PRED']= LISTA['CLAVECATASTRAL'].str[:10]

In [ ]:
m1=pd.merge(CAT, pred, on='CLAVE_PRED', how='left')
# pd.merge(LISTA, pred, on='CLAVE_PRED')
m2=pd.merge(E_PC, pred, on='CLAVE_PRED', how='left')

In [ ]:
LISTA.drop(columns=['Clave Catastral', 'CLAVE_PRED','CLAVE_PRED_y','CLAVE_PRED_x'], inplace=True)

In [ ]:
FINAL=pd.merge(LISTA, pred, on='CLAVE_PRED')

In [ ]:
FINAL=FINAL.sort_values('Número de Unidad', ascending=False)
# FINAL['CLAVECATASTRAL']=FINAL['Clave Catastral'].combine_first(LISTA['CLAVECATASTRAL'])

In [ ]:
FINAL=FINAL[['CLAVECATASTRAL','Número de Unidad', 'BDINTERNA_CONT_DIRECCION', 'CLAVEASENTAMIENTO',
       'CLAVECP', 'CLAVEMANZANA',
       'CLAVEZONA', 'CLAVESTATUS', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'ASENTAMIENTO_NR',
       'CLAVECP_NR', 'CLAVE_PRED', 'curt',
       'Latitude_clean', 'Longitude_clean',
       'COORDENADAS_RECAUDACION']]

In [ ]:
FINAL['CLAVEMANZANA_R'] = FINAL['CLAVECATASTRAL'].str[5:8]
FINAL['CLAVEZONA_R'] = FINAL['CLAVECATASTRAL'].str[3:5]

In [ ]:
FINAL.to_csv('CRUCE_ECATEPEC_CURTS_PRED_CATS.csv', encoding='ANSI')

In [ ]:
FINAL=pd.read_csv('CRUCE_ECATEPEC_CURTS_PRED_CATS.csv', encoding='ANSI')

In [ ]:
m2[['CLAVECATASTRAL', 'CLAVEASENTAMIENTO', 'CLAVECP', 'CLAVEMUNICIPIO',
       'CLAVEENTIDAD', 'CLAVEMANZANA', 'CLAVEZONA', 'CLAVESTATUS', 'DIRECCION',
       'VALORTERRENO', 'VALORTERRENOCOMUN', 'VALORCONSTPROP',
       'VALORCONSTCOMUN', 'SUPERFICIE', 'SUPERFICIETERRCOMUN',
       'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN', 'CLAVEANTERIOR', 'FECHAALTA', 'ASENTAMIENTO_NR', 'CLAVECP_NR', 'CLAVE_PRED', 'id_cat', 'curt', 'Latitude_clean', 'Longitude_clean', 'COORDENADAS_RECAUDACION']]

In [ ]:
pd.merge(CAT, pred, left_on='Clave Catastral', right_on='id_cat')
# pd.merge(E_PC, pred, left_on='CLAVECATASTRAL', right_on='id_cat').id_cat.str[3:5].value_counts()

In [ ]:
CATS=pd.read_csv('CRUCE_ECATEPEC_CURTS_PRED_CATS.csv', encoding='ANSI')

In [ ]:
CATS_CALLES=pd.read_csv('Downloads/ECATEPEC_CAT_QUERY_CALLES.txt', sep='\t', encoding='utf-8-sig')

In [ ]:
CATS_CALLES.drop_duplicates('key', inplace=True)

In [ ]:
CATS_CALLES.columns=CATS_CALLES.columns.str.upper()

CATS_CALLES.rename(columns={'FORMATTED_ADDRESS':'DIRECCION', 'GEOMETRY.LOCATION.LAT': 'LAT',
       'GEOMETRY.LOCATION.LNG': 'LON', 'PLUS_CODE.COMPOUND_CODE': 'PLUS_CODE'}, inplace=True)

In [ ]:
CATS_CALLES.columns=str('INTFIS_RS_FD_')+CATS_CALLES.columns

In [ ]:
CATS_CALLES['INTFIS_RS_FD_LAT']=CATS_CALLES['INTFIS_RS_FD_LAT'].astype(str).str.replace('SIN DATO','0', regex=False).astype(float).round(4)
CATS_CALLES['INTFIS_RS_FD_LON']=CATS_CALLES['INTFIS_RS_FD_LON'].astype(str).str.replace('SIN DATO','0', regex=False).astype(float).round(4)

In [ ]:
CATS_CALLES['COORDENADAS_RECAUDACION'] = CATS_CALLES['INTFIS_RS_FD_LAT'].astype(str).str[0:7].str.cat(CATS_CALLES['INTFIS_RS_FD_LON'].astype(str).str[0:8],', ') 

In [ ]:
P2=pd.merge(CATS_CALLES, CATS, on='COORDENADAS_RECAUDACION')

In [ ]:
CATS2= pd.read_csv('Downloads/CRUCE_ECATEPEC_GCP.csv', sep=',', encoding='utf-8-sig')

In [ ]:
LL['INTFIS_RS_FD_KEY']=LL.index

In [ ]:
P2=P2[['INTFIS_RS_FD_DIRECCION', 'INTFIS_RS_FD_PLACE_ID', 'INTFIS_RS_FD_TYPES',
       'INTFIS_RS_FD_LAT', 'INTFIS_RS_FD_LON', 'INTFIS_RS_FD_PLUS_CODE',
       'COORDENADAS_RECAUDACION','CLAVECATASTRAL',
       'Número de Unidad', 'BDINTERNA_CONT_DIRECCION', 'CLAVEASENTAMIENTO',
       'CLAVECP', 'CLAVEMANZANA', 'CLAVEZONA', 'CLAVESTATUS', 'DIRECCION',
       'VALORTERRENO', 'VALORTERRENOCOMUN', 'VALORCONSTPROP',
       'VALORCONSTCOMUN', 'SUPERFICIE', 'SUPERFICIETERRCOMUN',
       'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN', 'CLAVEANTERIOR', 'FECHAALTA',
       'ASENTAMIENTO_NR', 'CLAVECP_NR', 'CLAVE_PRED', 'curt', 'Latitude_clean',
       'Longitude_clean', 'CLAVEMANZANA_R', 'CLAVEZONA_R']]

In [ ]:
pp=pd.concat([P2,CATS2], axis=0)

In [ ]:
CATS.loc[~CATS['CLAVE_PRED'].isin(pp['CLAVE_PRED'])].to_csv('ECATEPEC_PARA_REV_GEO.txt', sep='\t', encoding='utf-8-sig')

In [ ]:
EGCP=pd.read_csv('GCP_AUT_PARCIAL_ECATEPEC.csv', encoding='utf-8-sig')

In [ ]:
EGCP['CLAVECATASTRAL']=EGCP['CLAVECATASTRAL'].astype(str).str.zfill(16)

In [ ]:
CC=CATS.loc[~CATS['CLAVECATASTRAL'].isin(EGCP['CLAVECATASTRAL'])]

In [ ]:
PP2=pd.merge(CC,PC, left_on='DIRECCION', right_on='BDINTERNA_CONT_DIRECCION')

In [ ]:
PN=pd.concat([EGCP, PP2], axis=0)

In [ ]:
CATS_CALLES['INTFIS_RS_FD_PARTIAL_MATCH'].notna().sum()

In [ ]:
EGCP['INTFIS_RS_FD_GEOMETRY.LOCATION_TYPE'].value_counts()

In [ ]:
EGCP.loc[EGCP['curt']=='193631083909901585500'][['CLAVECATASTRAL','curt','BDINTERNA_CONT_DIRECCION', 'DIRECCION','INTFIS_RS_FD_DIRECCION', 'INTFIS_RS_FD_PARTIAL_MATCH']]

In [ ]:
PC=pd.merge(LL,CATS_CALLES, on='INTFIS_RS_FD_KEY')

In [ ]:
PC=PC[['BDINTERNA_CONT_DIRECCION','INTFIS_RS_FD_PLACE_ID',
       'INTFIS_RS_FD_TYPES', 'INTFIS_RS_FD_LAT', 'INTFIS_RS_FD_LON', 'INTFIS_RS_FD_PLUS_CODE', 'INTFIS_RS_FD_DIRECCION', 'INTFIS_RS_FD_PARTIAL_MATCH', 'INTFIS_RS_FD_GEOMETRY.LOCATION_TYPE']]

In [ ]:
CATS[['CLAVECATASTRAL', 'Número de Unidad', 'CLAVEASENTAMIENTO', 'CLAVECP',
       'CLAVEMANZANA', 'CLAVEZONA', 'CLAVESTATUS', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'ASENTAMIENTO_NR', 'CLAVECP_NR',
       'CLAVE_PRED', 'curt', 'Latitude_clean', 'Longitude_clean',
       'CLAVEMANZANA_R', 'CLAVEZONA_R','DIRECCION']]

PPCATS=pd.merge(CATS,PC,left_on='DIRECCION', right_on='BDINTERNA_CONT_DIRECCION')

In [ ]:
CATS=CATS.drop(columns='BDINTERNA_CONT_DIRECCION' )
PC=PC.drop(columns='COORDENADAS_RECAUDACION')

In [ ]:
EC.rename(columns={'FORMATTED_ADDRESS':'DIRECCION', 'GEOMETRY.LOCATION.LAT': 'LAT',
       'GEOMETRY.LOCATION.LNG': 'LON', 'PLUS_CODE.COMPOUND_CODE': 'PLUS_CODE'}, inplace=True)
EGCP[['CLAVECATASTRAL','INTFIS_RS_FD_PLACE_ID',
       'INTFIS_RS_FD_TYPES', 'INTFIS_RS_FD_LAT', 'INTFIS_RS_FD_LON',
       'INTFIS_RS_FD_TIPO_VIALIDAD', 'INTFIS_RS_FD_PLUS_CODE', 'INTFIS_RS_FD_DIRECCION', 'CLAVECATASTRAL', 'INTFIS_RS_FD_PARTIAL_MATCH', 'INTFIS_RS_FD_GEOMETRY.LOCATION_TYPE']]

In [ ]:
PPCATS=pd.merge(CATS,PC,left_on='DIRECCION', right_on='BDINTERNA_CONT_DIRECCION', how='left')

In [ ]:
PPCATS=PPCATS[['CLAVECATASTRAL', 'Número de Unidad', 'CLAVEASENTAMIENTO',
       'CLAVECP', 'CLAVEMANZANA', 'CLAVEZONA', 'CLAVESTATUS', 'DIRECCION',
       'VALORTERRENO', 'VALORTERRENOCOMUN', 'VALORCONSTPROP',
       'VALORCONSTCOMUN', 'SUPERFICIE', 'SUPERFICIETERRCOMUN',
       'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN', 'CLAVEANTERIOR', 'FECHAALTA',
       'ASENTAMIENTO_NR', 'CLAVECP_NR', 'CLAVE_PRED', 'curt', 'Latitude_clean',
       'Longitude_clean', 'COORDENADAS_RECAUDACION', 'CLAVEMANZANA_R',
       'CLAVEZONA_R', 'INTFIS_RS_FD_PLACE_ID',
       'INTFIS_RS_FD_TYPES', 'INTFIS_RS_FD_LAT', 'INTFIS_RS_FD_LON',
       'INTFIS_RS_FD_PLUS_CODE', 'INTFIS_RS_FD_DIRECCION',
       'INTFIS_RS_FD_PARTIAL_MATCH', 'INTFIS_RS_FD_GEOMETRY.LOCATION_TYPE']]

In [ ]:
FFINAL=pd.merge(EGCP[[ 'CLAVECATASTRAL', 'INTFIS_RS_FD_PLACE_ID',
       'INTFIS_RS_FD_TYPES', 'INTFIS_RS_FD_LAT', 'INTFIS_RS_FD_LON',
       'INTFIS_RS_FD_TIPO_VIALIDAD', 'INTFIS_RS_FD_PLUS_CODE',
       'INTFIS_RS_FD_DIRECCION', 'INTFIS_RS_FD_PARTIAL_MATCH',
       'INTFIS_RS_FD_GEOMETRY.LOCATION_TYPE', 'BDINTERNA_CONT_DIRECCION']],PPCATS, on='CLAVECATASTRAL', how='outer')

In [ ]:
FFINAL.columns
# FFINAL
FFINAL=FFINAL[['CLAVECATASTRAL',  'BDINTERNA_CONT_DIRECCION',
       'Número de Unidad', 'CLAVEASENTAMIENTO', 'CLAVECP', 'CLAVEMANZANA',
       'CLAVEZONA', 'CLAVESTATUS', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'ASENTAMIENTO_NR', 'CLAVECP_NR',
       'CLAVE_PRED', 'curt', 'Latitude_clean', 'Longitude_clean',
       'COORDENADAS_RECAUDACION', 'CLAVEMANZANA_R', 'CLAVEZONA_R','INTFIS_RS_FD_PLACE_ID_x', 'INTFIS_RS_FD_TYPES_x',
       'INTFIS_RS_FD_LAT_x', 'INTFIS_RS_FD_LON_x',
       'INTFIS_RS_FD_TIPO_VIALIDAD', 'INTFIS_RS_FD_PLUS_CODE_x',
       'INTFIS_RS_FD_DIRECCION_x', 'INTFIS_RS_FD_PARTIAL_MATCH_x',
       'INTFIS_RS_FD_GEOMETRY.LOCATION_TYPE_x',
       'INTFIS_RS_FD_PLACE_ID_y', 'INTFIS_RS_FD_TYPES_y', 'INTFIS_RS_FD_LAT_y',
       'INTFIS_RS_FD_LON_y', 'INTFIS_RS_FD_PLUS_CODE_y',
       'INTFIS_RS_FD_DIRECCION_y', 'INTFIS_RS_FD_PARTIAL_MATCH_y',
       'INTFIS_RS_FD_GEOMETRY.LOCATION_TYPE_y']]
FFINAL.columns=FFINAL.columns.str.replace('_y', '_GEO_DOMS').str.replace('_x', '_GEO_COORDS')

In [ ]:
FFINAL['CLAVE_CONDOMINIOS']=FFINAL.CLAVECATASTRAL.str[10:]
FFINAL.loc[FFINAL.CLAVE_CONDOMINIOS!='000000','CONDOMINIOSxCLAVECAT'] = 1

p=FFINAL.curt.value_counts().reset_index()
p.loc[p['curt']>1, 'CURT_REPETIDOS']=1

p.rename(columns={'curt':'counts', 'index':'curt'}, inplace=True)

pd.merge(p,FFINAL, on='curt').to_csv('ECATEPEC_CRUCE_GCPx2.csv', encoding='utf-8-sig')

In [ ]:
CATS.loc[~CATS['CLAVE_PRED'].isin(PN['CLAVE_PRED'])].drop_duplicates('CLAVECATASTRAL')['CLAVE_PRED'].value_counts()

In [ ]:
CATS.loc[~CATS['CLAVE_PRED'].isin(PN['CLAVE_PRED'])].drop_duplicates('CLAVECATASTRAL').CLAVEZONA_R.value_counts()

In [ ]:
CC.loc[CC['DIRECCION'].isin(PC['BDINTERNA_CONT_DIRECCION'])]

In [ ]:
BF=pd.read_csv('ECATEPEC_CRUCE_GCPx2.csv', encoding='utf-8-sig')

In [ ]:
DBF=pd.read_excel('Downloads/ECATEPEC GC203T06 (nov. 2021).xlsx')

In [ ]:
DBF['EDIFICIO']=DBF['EDIFICIO'].str.zfill(2)
DBF['DEPTO']=DBF['DEPTO'].str.zfill(4)

In [ ]:
DBF['CLAVECATASTRAL']=DBF.MUNICIPIO.astype(str).str.zfill(3).str.cat(DBF.ZONA.astype(str).str.zfill(2)).str.cat(DBF.MANZANA.astype(str).str.zfill(3)).str.cat(DBF.LOTE.astype(str).str.zfill(2)).str.cat(DBF.EDIFICIO).str.cat(DBF.DEPTO)

In [ ]:
DBF['CLAVECATASTRAL'].value_counts()
DBF.loc[DBF['CLAVECATASTRAL']=='0942000622000000']

In [ ]:
DBF['CLAVE_PRED'] = DBF['CLAVECATASTRAL'].str[0:10]


In [ ]:
BF.CLAVE_PRED=BF.CLAVE_PRED.astype(str).str.zfill(10)
# BF.loc[BF['CLAVE_PRED']=='942000622']

In [ ]:
## DBF.iloc[818691]['MUNICIPIO']=98


In [ ]:

BF=BF.drop(columns=['Unnamed: 0']).drop_duplicates()

In [ ]:
BF.loc[BF.CLAVECATASTRAL=='0942509401100101'][['INTFIS_RS_FD_DIRECCION_GEO_DOMS','INTFIS_RS_FD_DIRECCION_GEO_COORDS','BDINTERNA_CONT_DIRECCION','DIRECCION','curt', 'INTFIS_RS_FD_TYPES_GEO_DOMS']]

In [ ]:
BF.columns

In [ ]:
BF.drop_duplicates(['CLAVECATASTRAL','INTFIS_RS_FD_DIRECCION_GEO_DOMS','INTFIS_RS_FD_DIRECCION_GEO_COORDS','BDINTERNA_CONT_DIRECCION','DIRECCION'],inplace=True)

In [ ]:
BF.columns

In [ ]:
BF2=pd.merge(BF,DBF[['CLAVECATASTRAL','USO']].drop_duplicates('CLAVECATASTRAL'),on='CLAVECATASTRAL', how='left')

In [ ]:
from padagua_2021.Geolocalizacion.concatenacion_qr.qr_generation import coords_col, generate_qrs, generate_urls  

In [ ]:
def creacion_urls(base, term=''):
    base = base.reset_index()
    base["ORDEN"] = base.index + 1

    # Cambios de formato
    latlong_col = (
        base['INTFIS_RS_FD_LAT'+term].astype(str)
        + ", "
        + base['INTFIS_RS_FD_LON'+term].astype(str)
    )
    latlong_col = latlong_col.apply(lambda latlon: None if "nan" in latlon else latlon)

    base["COORDS"+str(term)] = latlong_col
    complete_section = base[~base["COORDS"+str(term)].isna()]

    assert sum(complete_section["COORDS"+str(term)].isna()) == 0

    complete_section.loc[:, "INTFIS_RS_FD_URLS"+str(term)]= generate_urls(
        complete_section["COORDS"+str(term)]+term
    )

    # n = (
    #     base.groupby(["INTFIS_RS_FD_CALLE"+str(term), "INTFIS_RS_FD_COLONIA"+str(term)])
    #     .sum()
    #     .reset_index()["INTFIS_RS_FD_COLONIA"+str(term)]
    #     .value_counts()
    # )
    # filtro = base["INTFIS_RS_FD_COLONIA"+str(term)].isin(n[n > 1].index)
    # base.loc[filtro, "INTFIS_RS_FD_URLS"+str(term)] = (
    #     "https://www.google.com/maps/search/?api=1&query=" + latlong_col[filtro]
    # )

    

    base.loc[:, "INTFIS_RS_FD_URLS"+str(term)] = None
    base = base[base["COORDS"+str(term)].isna()].append(complete_section)

    logging.info(f"Tamaño final de la base: {base.shape}")

    # Nos aseguramos que todos los URLS tengan un valor de ORDEN unico
    assert len(
        complete_section[(~complete_section['INTFIS_RS_FD_URLS'+str(term)].isna())].ORDEN.unique()
    ) == sum((~complete_section['INTFIS_RS_FD_URLS'+term].isna()))

    base.drop(columns=["COORDS"+str(term)], inplace=True)
    # base.to_csv(r'E:\RecepcionInfo\TOTALES_BLOQUE1\Naucalpan de Juarez\Naucalpan_parcial.csv', encoding='utf-8-sig')
    return base

In [ ]:
GCP0=pd.read_csv('Downloads/ECATEPEC_PARA_REV_GEO.txt', sep='\t', encoding='utf-8-sig')

In [ ]:
GCP0.loc[~GCP0['CLAVECATASTRAL'].isin(pred['id_cat'])]

In [ ]:
GCP0=GCP0[['CLAVECATASTRAL', 'Número de Unidad', 'BDINTERNA_CONT_DIRECCION',
       'CLAVEASENTAMIENTO', 'CLAVECP', 'CLAVEMANZANA', 'CLAVEZONA',
       'CLAVESTATUS', 'DIRECCION_CATS', 'VALORTERRENO', 'VALORTERRENOCOMUN',
       'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'ASENTAMIENTO_NR', 'CLAVECP_NR',
       'CLAVE_PRED', 'curt', 'Latitude_clean', 'Longitude_clean',
       'COORDENADAS_RECAUDACION', 'CLAVEMANZANA_R', 'CLAVEZONA_R',
       'DIRECCION',
       'place_id', 'types', 'geometry.location.lat', 'geometry.location.lng',
       'geometry.location_type', 'plus_code.compound_code']]

In [ ]:
GCP0.rename(columns={'place_id':'INTFIS_RS_FD_PLACE_ID', 'geometry.location.lat': 'INTFIS_RS_FD_LAT',
        'geometry.location.lng': 'INTFIS_RS_FD_LON', 'plus_code.compound_code': 'INTFIS_RS_FD_PLUS_CODE', 'DIRECCION':'INTFIS_RS_FD_DIRECCION'}, inplace=True)

In [ ]:
GCP0.columns

In [ ]:
pbf2=BF2.loc[(BF2['CLAVECATASTRAL'].isin(GCP0['CLAVECATASTRAL'])) & (BF2['INTFIS_RS_FD_DIRECCION_GEO_COORDS'].isna())]

In [ ]:
pbf2=pbf2[pbf2.columns[~pbf2.columns.str.contains('_GEO_COORDS')]]

In [ ]:
pbf2.columns[~pbf2.columns.isin(GCP0.columns)]

In [ ]:
pbf2=pbf2[['counts', 'CURT_REPETIDOS', 'DIRECCION', 'INTFIS_RS_FD_TIPO_VIALIDAD',
       'INTFIS_RS_FD_PLACE_ID_GEO_DOMS', 'INTFIS_RS_FD_TYPES_GEO_DOMS',
       'INTFIS_RS_FD_LAT_GEO_DOMS', 'INTFIS_RS_FD_LON_GEO_DOMS',
       'INTFIS_RS_FD_PLUS_CODE_GEO_DOMS', 'INTFIS_RS_FD_DIRECCION_GEO_DOMS',
       'INTFIS_RS_FD_PARTIAL_MATCH_GEO_DOMS',
       'INTFIS_RS_FD_GEOMETRY.LOCATION_TYPE_GEO_DOMS', 'CLAVE_CONDOMINIOS',
       'CONDOMINIOSxCLAVECAT', 'USO','CLAVECATASTRAL']]

In [ ]:
BF3=pd.merge(pbf2, GCP0, on='CLAVECATASTRAL')

In [ ]:
pbf3=BF2.loc[~(BF2['CLAVECATASTRAL'].isin(BF3['CLAVECATASTRAL']))]

In [ ]:
BF4=pd.concat([BF3, pbf3], axis=0)

In [ ]:
CAT.CLAVECATASTRAL.value_counts()

In [ ]:
CAT.loc[~CAT['Clave Catastral'].isin(BF4.CLAVECATASTRAL)]['Clave Catastral'].value_counts()

In [ ]:
CAT['ZONA']=CAT['Clave Catastral'].str[3:5]

In [ ]:
ZONAS=pd.read_excel('Downloads/Conteo Zonas a favor 80.xlsx')

In [ ]:
ZONAS['ZONA']=ZONAS.Zona.astype(str).str.replace('.0','', regex=False).str.zfill(5).str[3:]

In [ ]:
CAT=CAT.loc[CAT['ZONA'].isin(ZONAS['ZONA'])]

In [ ]:
len(CAT['ZONA'].value_counts())

In [ ]:
CAT['CLAVE_PRED'] = CAT['Clave Catastral'].str[0:10]

In [ ]:
E_PC['CLAVE_PRED'] = E_PC['CLAVECATASTRAL'].str[0:10]

In [ ]:
CC

In [ ]:
CC=CAT.drop_duplicates('Clave Catastral')

In [ ]:
# E_PC.CLAVECATASTRAL.value_counts()
PRED_CAT=pd.merge(E_PC, CC, right_on='Clave Catastral', left_on='CLAVECATASTRAL')

In [ ]:
BF4.loc[~BF4['CLAVECATASTRAL'].isin(PRED_CAT['CLAVECATASTRAL'])]
PC=PRED_CAT.loc[~PRED_CAT['CLAVECATASTRAL'].isin(BF4['CLAVECATASTRAL'])]

In [ ]:
PC=PC[PC.columns[PC.columns.isin(BF4.columns)]]

In [ ]:
CC.groupby(['ZONAS','Número de Unidad']).sum()
CAT.loc[CAT['ZONA']=='19']['Número de Unidad'].value_counts().sum()

In [ ]:
CAT=CAT.loc[CAT['ZONA']!='11']

In [ ]:
nn=CAT['Clave Catastral'].value_counts()

In [ ]:
n=CAT.groupby('Clave Catastral').min()

In [ ]:
PCC=CC.loc[~CC['Clave Catastral'].isin(BF5['CLAVECATASTRAL'])]

In [ ]:
BF['CLAVECATASTRAL']=BF['CLAVECATASTRAL'].astype(str).str.zfill(16)

In [ ]:
E_PC.loc[~E_PC['CLAVECATASTRAL'].isin(PRED_CAT['CLAVECATASTRAL'])]

In [ ]:
pd.merge(CC,DBF[['CLAVECATASTRAL','USO']].drop_duplicates('CLAVECATASTRAL'),on='CLAVECATASTRAL', how='left')

In [ ]:
pd.merge(PC, pred,  left_on='CLAVECATASTRAL',right_on='id_cat')

In [ ]:
CC.loc[~CC['Clave Catastral'].isin(BF4['CLAVECATASTRAL'])]

In [ ]:
PCC.rename(columns={'Clave Catastral':'CLAVECATASTRAL','Domicilio del Inmueble':'BDINTERNA_CONT_DIRECCION'}, inplace=True)

In [ ]:
BF5=pd.concat([PC,BF4], axis=0)

In [ ]:
BF6=pd.concat([PCC,BF5], axis=0)

In [ ]:
DBF.CLAVECATASTRAL.value_counts()
DBF.drop_duplicates('CLAVECATASTRAL', inplace=True)

In [ ]:
BF6.drop_duplicates('CLAVECATASTRAL', inplace=True)

In [ ]:
BF6=BF6.loc[BF6['CLAVECATASTRAL'].isin(CC['Clave Catastral'])]

In [ ]:
BF7=pd.merge(BF6.drop_duplicates('CLAVECATASTRAL'),DBF[['CLAVECATASTRAL','USO','CLAVE_PRED']].drop_duplicates('CLAVECATASTRAL'),on='CLAVECATASTRAL', how='left')

In [ ]:
BF7.loc[BF7['CLAVECATASTRAL'].isin(CC['Clave Catastral'])]
# CC.loc[CC['Clave Catastral'].isin(BF7['CLAVECATASTRAL'])]

In [ ]:
BF6.columns[BF6.columns.isin(DBF.columns)]
BF6.drop(columns=['ZONA', 'CLAVE_PRED', 'USO'], inplace=True)

In [ ]:
BF7.loc[~BF7['CLAVECATASTRAL'].isin(CC['Clave Catastral'])].drop_duplicates('CLAVECATASTRAL')

In [ ]:
CAT['Clave Catastral'].value_counts()

In [ ]:
E_PC.loc[E_PC['CLAVEZONA'].astype(str).str.zfill(2).isin(CAT['ZONA'])]

In [ ]:
BF7.columns

BF7['CLAVE_CONDOMINIOS']=BF7.CLAVECATASTRAL.str[10:]
BF7.loc[BF7.CLAVE_CONDOMINIOS!='000000','CONDOMINIOSxCLAVECAT'] = 1

p=BF7.curt.value_counts().reset_index()
p.loc[p['curt']>1, 'CURT_REPETIDOS']=1

p.rename(columns={'curt':'counts', 'index':'curt'}, inplace=True)

BF7=pd.merge(p,BF7, on='curt', how='right')

In [ ]:
BF7.USO.value_counts()

In [ ]:
DBF

In [ ]:
BF7.to_csv('NAUCALPAN_CRUCE_DBF_PRED_GCP.csv', encoding='utf-8-sig')
BF7.loc[BF7['USO']=='H'][['CLAVECATASTRAL','USO']].to_csv('NAUCALPAN_CRUCE_DBF_PRED_GCP_IDS.csv', encoding='utf-8-sig')

In [ ]:
BF7= pd.read_csv('NAUCALPAN_CRUCE_DBF_PRED_GCP.csv', encoding='utf-8-sig')

In [ ]:
pred

In [ ]:
BF7.loc[BF7['CLAVECATASTRAL']=='0940815622000000']['DIRECCION'][22849]
# pred.loc[pred['id_cat']=='0940815622000000']

In [ ]:
BF7['INTFIS_RS_FD_GEOMETRY_LOCATION_TYPE_GEO_COORDS'].value_counts()

In [ ]:
ad=pd.merge(E_PP.groupby('CLAVECATASTRAL')['STATUS'].count().reset_index().rename(columns={'STATUS':'STATUS_counts'}), E_PP, on='CLAVECATASTRAL')
ad.EJERCICIO.min()
ad.loc[(ad['EJERCICIO']==2022) & (ad['STATUS']=='Adeudo'), 'limite adeudo']= 'adeudo máximo'
ad.loc[(ad['EJERCICIO']==2022)& (ad['STATUS']=='Pagada'), 'limite pago']= 'pago máximo'
ad.loc[(ad['EJERCICIO']==2007)& (ad['STATUS']=='Adeudo'),'limite adeudo']= 'adeudo máximo'
ad.loc[(ad['EJERCICIO']==2007)& (ad['STATUS']=='Pagada'), 'limite pago']= 'pago máximo'

In [ ]:
ad.loc[(ad['EJERCICIO']<2022)& (ad['STATUS']=='Pagada'), 'limite pago']= 'pago parcial'
ad.loc[(ad['EJERCICIO']==2022)& (ad['STATUS']=='Pagada'), 'limite pago']= 'pago máximo'

In [ ]:
NN=pd.merge(BF7.loc[BF7['USO']=='H'], ad[['CLAVECATASTRAL','EJERCICIO','limite adeudo','limite pago']], on='CLAVECATASTRAL')


In [ ]:
PBB=pd.merge(DBF, PRED_CAT, on='CLAVECATASTRAL').drop_duplicates('CLAVECATASTRAL')[['CLAVECATASTRAL','USO']]
PBB.loc[PBB['USO']=='H'].to_csv('NAUCALPAN_CRUCE_DBF_PRED_GCP_IDS_2.csv', encoding='utf-8-sig')

In [ ]:
PBB.loc[PBB['USO']=='H']['CLAVECATASTRAL'].to_csv('NAUCALPAN_CRUCE_DBF_PRED_GCP_IDS_2.csv', encoding='utf-8-sig')

In [ ]:
BF7.loc[BF7['CLAVECATASTRAL'].isin(PBB2['CLAVECATASTRAL'])].INTFIS_RS_FD_DIRECCION_GEO_COORDS.notna().sum()

In [ ]:
PRED_CAT.CLAVECATASTRAL.value_counts()

PBB2=pd.merge(PBB.loc[PBB['USO']=='H'], E_PP[['CLAVECATASTRAL','EJERCICIO','STATUS']], on='CLAVECATASTRAL')

PBB2.loc[(PBB2['EJERCICIO']<2022) & (PBB2['STATUS'] =='Pagada')].CLAVECATASTRAL.value_counts()

In [ ]:
E_PC['CLAVE_PRED']=E_PC['CLAVE_PRED']+'000000'

In [ ]:
E_PC.groupby('').count.reset_index

In [ ]:
PRUEBA=pd.merge(E_MCAT, left_on='CLAVE_PRED', right_on='Clave Catastral'oupby('CLAVE_PRED').coynXs()..reset_index)().

In [ ]:
MUESTRA=BF7.loc[(BF7['Número de Unidad']>1) & (BF7['INTFIS_RS_FD_DIRECCION_GEO_COORDS'].notna()) ]
MUESTRA[MUESTRA.columns[~MUESTRA.columns.str.contains('_x|_y|_GEO_DOMS')]].to_csv('MUESTRAS_GCP_catastro.csv', encoding='utf-8-sig')

# Código funcional

### Nota : 
La preparación de las bases para las queries de gcp, es distinta:
Para aquellas que tienen curt, se genera el punto a buscar
Para aquellas que tienen dirección se hace limpieza de domicilios y se prepara la base para la búsqueda por domicilio (lleva más tiempo)
     
     
     *Desde la limpieza, también se escoge el domcilio más correcto, en estos casos donde exiten 2 domcilios por CAT (1-2 horas)*
    
    
     *La parte del querie, 6 horas por 100 mil aprox *




In [ ]:
CAT_N=pd.read_excel('Downloads/Ecatepec claves  1 a 1 depurado.xlsx')

In [ ]:
CAT_N

In [ ]:
PRED_CAT_N=pd.merge(CAT_N, E_PC, left_on='Clave Catastral', right_on='CLAVECATASTRAL', how='outer')
PRED_CAT_N['CLAVECATASTRAL']=PRED_CAT_N.CLAVECATASTRAL.combine_first(PRED_CAT_N['Clave Catastral'])
PRED_CAT_CURTS=pd.merge(pred, PRED_CAT_N, left_on='id_cat', right_on='CLAVECATASTRAL', how='right')
PRED_CAT_CURTS['CLAVECATASTRAL']=PRED_CAT_CURTS.CLAVECATASTRAL.combine_first(PRED_CAT_CURTS['id_cat'])
PRED_CAT_CURT_DBF=pd.merge(PRED_CAT_CURTS, DBF[['USO','CLAVECATASTRAL']].drop_duplicates('CLAVECATASTRAL'), on='CLAVECATASTRAL', how='left')

In [ ]:
PRED_CAT_CURT_DBF=PRED_CAT_CURT_DBF.drop_duplicates('CLAVECATASTRAL')

In [ ]:
# CAT_N['Clave Catastral']
# DBF.columns
PCCDF=PRED_CAT_CURT_DBF.loc[(PRED_CAT_CURT_DBF['curt'].notna())| (PRED_CAT_CURT_DBF['DIRECCION'].notna())| (PRED_CAT_CURT_DBF['Domicilio de inmueble'].notna())]

In [ ]:
ads=pd.read_csv('Ecatepec_parcial.csv', encoding='utf-8-sig')

In [ ]:
ads[['CLAVECATASTRAL', 'INTFIS_RS_FD_GEO_CLAVECATASTRAL_PREDIO',
       'CLAVECATASTRAL_RECAUDACION']]
ads=ads.loc[ads['CLAVECATASTRAL']=='0000000000000nan'].drop(columns=['CLAVECATASTRAL', 'INTFIS_RS_FD_GEO_CLAVECATASTRAL_PREDIO',
       'CLAVECATASTRAL_RECAUDACION'])

In [ ]:
ads=ads.loc[~ads['INTFIS_RS_FD_PLACE_ID'].isin(GCP0['INTFIS_RS_FD_PLACE_ID'])]

In [ ]:
# DIRS.loc[DIRS['DIRECCION'].astype(str).str.contains('N°')]['DIRECCION']
DIRS['DIRECCION']=DIRS['DIRECCION'].astype(str)

In [ ]:
DIRS=PCCDF.loc[(PCCDF['Domicilio de inmueble'].notna()) & (PCCDF['curt'].isna())]
DIRS['MANZANA_PREDIAL']= DIRS['DIRECCION'].str.split('MZ|NO.', expand=True)[1]
DIRS['MANZANA_PREDIAL']= 'MZ'+DIRS['MANZANA_PREDIAL']
DIRS['DIRECCION_0']=DIRS.apply(lambda row: row['DIRECCION'].replace(str(row['MANZANA_PREDIAL']), ""),axis=1).str.replace(',','').str.strip(' ').str.replace(r'\s+', ' ', regex=True)
DIRS['LOTE_PREDIAL']= DIRS['DIRECCION_0'].str.split('LT', expand=True)[1]
DIRS['LOTE_PREDIAL']= 'LT'+DIRS['LOTE_PREDIAL']
DIRS['DIRECCION_0']=DIRS.apply(lambda row: row['DIRECCION_0'].replace(str(row['LOTE_PREDIAL']), ""),axis=1).str.replace(',','').str.strip(' ').str.replace(r'\s+', ' ', regex=True)

# # 
DIRS['N_EXT_PREDIAL']=DIRS['DIRECCION_0'].str.replace(r'\s+', ' ').str.strip().str.split(' NO.|,|N°|Nª').str[1].str.findall(r"(\d{1}\-?\d{0,4})").str[0]


In [ ]:
DIRS=DIRS[['DIRECCION','N_EXT_PREDIAL','Domicilio de inmueble','Domicilio del inmueble construido','Clave Catastral', 'Municipio', 'Zona',
       'Manzana', 'Lote', 'Edificio', 'Departamento', 'Año de construcción',
       'Estado de la Construcción', 'Niveles de Construcción',
       'Cantidad de veces que se repite clave catastral', 'CLAVECATASTRAL',
       'CLAVEASENTAMIENTO', 'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD',
       'CLAVEMANZANA', 'CLAVEZONA', 'CLAVESTATUS', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'LATITUD', 'LONGITUD', 'FECHAALTA', 'ASENTAMIENTO_NR',
       'CLAVECP_NR', 'USO']]
DIRS['COLONIA_catastro']=DIRS['Domicilio del inmueble construido'].str.split('COLONIA|CALLE|NUMERO EXTERIOR|EDIFICIO|DEPARTAMENTO|NUMERO INTERIOR').str[1]
DIRS['CALLE_catastro']=DIRS['Domicilio del inmueble construido'].str.split('COLONIA|CALLE|NUMERO EXTERIOR|EDIFICIO|DEPARTAMENTO|NUMERO INTERIOR').str[2]
DIRS['NUMERO_EXTERIOR_catastro']=DIRS['Domicilio del inmueble construido'].str.split('COLONIA|CALLE|NUMERO EXTERIOR|EDIFICIO|DEPARTAMENTO|NUMERO INTERIOR').str[3]
DIRS['EDIFICIO_catastro']=DIRS['Domicilio del inmueble construido'].str.split('COLONIA|CALLE|NUMERO EXTERIOR|EDIFICIO|DEPARTAMENTO|NUMERO INTERIOR').str[4]
DIRS['DEPARTAMENTO_catastro']=DIRS['Domicilio del inmueble construido'].str.split('COLONIA|CALLE|NUMERO EXTERIOR|EDIFICIO|DEPARTAMENTO|NUMERO INTERIOR').str[5]
DIRS['NUMERO_INTERIOR_catastro']=DIRS['Domicilio del inmueble construido'].str.split('COLONIA|CALLE|NUMERO EXTERIOR|EDIFICIO|DEPARTAMENTO|NUMERO INTERIOR').str[6]

DIRS['MANZANA_LOTE_catastro']= DIRS['Domicilio de inmueble'].str.split('MZ|MZA', expand=True)[1]
DIRS['MANZANA_LOTE_catastro']= 'MZ'+DIRS['MANZANA_LOTE_catastro']
DIRS['CALLE_catastro_0']=DIRS.apply(lambda row: row['Domicilio de inmueble'].replace(str(row['MANZANA_LOTE_catastro']), ""),axis=1).str.replace(',','').str.strip(' ').str.replace(r'\s+', ' ', regex=True)

DIRS['NUMERO_EXTERIOR_catastro']=DIRS['NUMERO_EXTERIOR_catastro'].str.extract('(\d+)')


DIRS['N_EXT_catastro_0']=DIRS['CALLE_catastro_0'].str.replace(r'\s+', ' ').str.strip().str.split(' NO.|N°|Nº').str[1].str.findall(r"(\d{1}\-?\d{0,4})").str[0]
DIRS['CALLE_catastro_0']=DIRS['CALLE_catastro_0'].astype(str)
DIRS['CALLE_catastro_0']=DIRS.apply(lambda row: row['CALLE_catastro_0'].replace(str(row['N_EXT_catastro_0']), ""),axis=1).str.replace(',','').str.strip(' ').str.replace(r'\s+', ' ', regex=True)
DIRS['CALLE_catastro_0']=DIRS['CALLE_catastro_0'].str.replace('N°','').str.replace('Nº','')

DIRS['N_EXT_catastro_0']
DIRS['CALLE_catastro_0'].str.replace(r'\s+', ' ').str.strip().str.split(' NO.|N°|Nº').str[1].str.findall(r"(\d{1}\-?\d{0,4})").str[0][308275]
DIRS['CALLE_catastro_0'].astype(str)[308275]
DIRS.apply(lambda row: row['CALLE_catastro_0'].replace(str(row['N_EXT_catastro_0']), ""),axis=1).str.replace(',','').str.strip(' ').str.replace(r'\s+', ' ', regex=True)[308275]
DIRS['N_EXT_catastro_0'][308275]
# DIRS['N_EXT_PREDIAL']=DIRS['N_EXT_PREDIAL'].str.extract('(\d+)')

# DIRS['CALLE_PRED']= DIRS['DIRECCION_0'].str.split('N°| NO.').str[0].str.replace('CALLE','').str.replace('AV.','', regex=False).str.replace('SIN','')
# DIRS.loc[(DIRS['N_EXT_PREDIAL'].astype(str).str.strip() !=DIRS['NUMERO_EXTERIOR_catastro'].str.strip().astype(str)) & (DIRS['N_EXT_PREDIAL'].notna())][['DIRECCION','DIRECCION_0','CALLE_PRED','N_EXT_PREDIAL','MANZANA_PREDIAL','LOTE_PREDIAL','Domicilio de inmueble','Domicilio del inmueble construido','COLONIA_catastro','CALLE_catastro','NUMERO_EXTERIOR_catastro','CALLE_catastro_0','N_EXT_catastro_0']]
# DIRS.loc[(DIRS['CALLE_PRED']==' ')|(DIRS['CALLE_PRED']=='') | (DIRS['CALLE_PRED'].str.contains('SIN NOMBRE'))| (DIRS['CALLE_PRED'].str.contains('SIN')),'CALLE_PRED']=float('NaN')


# DIRS[['DIRECCION','N_EXT_PREDIAL','Domicilio de inmueble','Domicilio del inmueble construido','COLONIA_catastro','CALLE_catastro','NUMERO_EXTERIOR_catastro','CALLE_catastro_0']]

DIRS.loc[(DIRS['CALLE_catastro_0']==' ') |(DIRS['CALLE_catastro_0']=='') | (DIRS['CALLE_catastro_0'].str.contains('SIN NOMBRE')),'CALLE_catastro_0']=float('NaN')
DIRS.loc[(DIRS['CALLE_catastro']==' ')|(DIRS['CALLE_catastro']=='') | (DIRS['CALLE_catastro'].str.contains('SIN NOMBRE')),'CALLE_catastro']=float('NaN')

DIRS['CALLE_catastro_final']=DIRS['CALLE_catastro'].combine_first(DIRS['CALLE_catastro_0']).str.replace('CALLE','').str.replace('N°','', regex=False).str.replace('AV.','', regex=False)
DIRS['NUMERO_EXTERIOR_catastro_final']=DIRS['NUMERO_EXTERIOR_catastro'].combine_first(DIRS['N_EXT_catastro_0'])


DIRS.loc[(DIRS['CALLE_catastro_final'].astype(str).str.strip() ==DIRS['CALLE_PRED'].str.strip().astype(str)) & (DIRS['N_EXT_PREDIAL'].astype(str).str.strip() !=DIRS['NUMERO_EXTERIOR_catastro_final'].str.strip().astype(str))&(DIRS['NUMERO_EXTERIOR_catastro_final'].notna())&(DIRS['N_EXT_PREDIAL'].notna())][['DIRECCION','DIRECCION_0','CALLE_PRED','N_EXT_PREDIAL','Domicilio de inmueble','Domicilio del inmueble construido','COLONIA_catastro','CALLE_catastro','NUMERO_EXTERIOR_catastro','NUMERO_EXTERIOR_catastro_final','CALLE_catastro_0','N_EXT_catastro_0','CALLE_catastro_final']]

DIRS.columns
DIRS['DOMICILIO_CATASTRO']= DIRS['CALLE_catastro_0'].str.cat(DIRS[['NUMERO_EXTERIOR_catastro_final','MANZANA_LOTE_catastro', 'COLONIA_catastro','EDIFICIO_catastro','DEPARTAMENTO_catastro', 'NUMERO_INTERIOR_catastro']].astype(str), ' ').str.replace('nan','')

DIRS.loc[DIRS['DOMICILIO_CATASTRO'].str.contains('0 0'),'DOMICILIO_CATASTRO'] =float('NaN')

DIRS['DIRECCION_FINAL']=DIRS['DOMICILIO_CATASTRO'].combine_first(DIRS['DIRECCION'])

In [ ]:
DIRS.loc[(DIRS['DIRECCION_FINAL']!='nan') &(~DIRS['DIRECCION_FINAL'].str.contains('0 0'))][['CLAVECATASTRAL',  'DIRECCION_FINAL']].to_csv('BUSQUEDA_ECATEPEC.csv', encoding='utf-8-sig')

In [ ]:
DIRS_CALLES_0=DIRS.loc[(DIRS['DIRECCION_FINAL']!='nan') &(~DIRS['DIRECCION_FINAL'].str.contains('0 0'))][['CLAVECATASTRAL',  'DIRECCION_FINAL']]

In [ ]:
DIRS_CALLES_0['INTFIS_RS_FD_KEY']=DIRS_CALLES_0.reset_index().index

Acuerdate de quitat el index para evitar la columna Unnamed

In [ ]:
DIRS_CALLES=pd.read_csv('Downloads/ECATEPEC_CAT_QUERY_CALLES (1).txt', sep='\t', encoding='utf-8-sig', index=False)

In [ ]:
DIRS_CALLES.columns=DIRS_CALLES.columns.str.upper()
DIRS_CALLES.rename(columns={'FORMATTED_ADDRESS': 'DIRECCION', 'GEOMETRY.LOCATION.LNG':'LON', 'GEOMETRY.LOCATION.LAT':'LAT', 'PLUS_CODE.COMPOUND_CODE':'PLUS_CODE', 'GEOMETRY.LOCATION_TYPE':'GEOM_TYPE'}, inplace=True)
DIRS_CALLES.columns=str('INTFIS_RS_FD_')+DIRS_CALLES.columns

In [ ]:
DIRS_CALLES=DIRS_CALLES.drop_duplicates('INTFIS_RS_FD_KEY', keep='last')

In [ ]:
# DIRS_CALLES.columns
DIRS_CALLES_N=pd.merge(DIRS_CALLES,DIRS_CALLES_0, on='INTFIS_RS_FD_KEY')

In [ ]:
PRED_CAT_FINAL_N=pd.concat([PRED_CAT_FINAL, DIRS_2_N], axis=0)

In [ ]:
PRED_CAT_FINAL_N[['CLAVECATASTRAL_RECAUDACION','CLAVECATASTRAL','CATASTRO_MUNICIPIO',
       'CATASTRO_ZONA', 'CATASTRO_MANZANA', 'CATASTRO_LOTE',
       'CATASTRO_EDIFICIO', 'CATASTRO_DEPARTAMENTO',
       'CATASTRO_ANIO_CONSTRUCCION', 'CATASTRO_ESTADO_CONSTRUCCION',
       'CATASTRO_NIVELES_CONSTRUCCION', 'CATASTRO_DOMICILIO_INMUEBLE',
       'CATASTRO_DOMICILIO_INMUEBLE_CONSTRUIDO',
       'CATASTRO_CANTIDAD_REPITE_CVECATASTRAL','BDINTERNA_CONT_NOMBRE_COMPLETO', 'BDINTERNA_CONT_RFC',
       'BDINTERNA_CONT_EMAIL', 'BDINTERNA_CONT_DIRECCION',
       'BD_INTERNA_FUENTE_ORIGEN','DIRECCION_FINAL','CATASTRO_DOMICILIO_CATASTRO','INTFIS_RS_FD_DIRECCION', 'INTFIS_RS_FD_PLACE_ID', 'INTFIS_RS_FD_TYPES',
       'INTFIS_RS_FD_LAT', 'INTFIS_RS_FD_LON', 'INTFIS_RS_FD_GEOM_TYPE',
       'INTFIS_RS_FD_PLUS_CODE',
       'ID_MUNICIPIO', 'ORDEN',
        'CURT', 'ESTATUS_2007', 'ESTATUS_2008',
       'ESTATUS_2009', 'ESTATUS_2010', 'ESTATUS_2011', 'ESTATUS_2012',
       'ESTATUS_2013', 'ESTATUS_2014', 'ESTATUS_2015', 'ESTATUS_2016',
       'ESTATUS_2017', 'ESTATUS_2018', 'ESTATUS_2019', 'ESTATUS_2020',
       'ESTATUS_2021', 'ESTATUS_2022', 'VALORCATASTRAL_2007',
       'VALORCATASTRAL_2008', 'VALORCATASTRAL_2009', 'VALORCATASTRAL_2010',
       'VALORCATASTRAL_2011', 'VALORCATASTRAL_2012', 'VALORCATASTRAL_2013',
       'VALORCATASTRAL_2014', 'VALORCATASTRAL_2015', 'VALORCATASTRAL_2016',
       'VALORCATASTRAL_2017', 'VALORCATASTRAL_2018', 'VALORCATASTRAL_2019',
       'VALORCATASTRAL_2020', 'VALORCATASTRAL_2021', 'VALORCATASTRAL_2022',
       'USO', 'PREDIAL_PROPIETARIO_NOMBRE', 'PREDIAL_PROPIETARIO_CALLE',
       'PROPIETARIO_ASENTAMIENTO', 'PROPIETARIO_CP', 'NOMBRE_MUNICIPIO',
       'FECHA_CORTE', 'PAGO_AL_MENOS_UNO']].to_csv('BASE_PREDIAL_CATASTRO_ECATEPEC.csv', encoding='utf-8-sig', index=False)

In [ ]:
tipo_dir="INTFIS_RS_FD_"
base_folder= 'C:\\Users\\dlara\\'
PRED_CAT_FINAL_N=limpieza_norm_dir(PRED_CAT_FINAL_N, tipo_dir, debug=False)
PRED_CAT_FINAL_N=load_dict(PRED_CAT_FINAL_N,base_folder, tipo_dir, tipo_dict="CALLE", score=80)
PRED_CAT_FINAL_N=load_dict(PRED_CAT_FINAL_N,base_folder, tipo_dir, tipo_dict="COLONIA", score=80)
PRED_CAT_FINAL_N=full_doms(PRED_CAT_FINAL_N,tipo_dir)

In [ ]:
# hallar aquellas que por place id se repiten dentro de la base de predial catastro o nómina. 
# cortar la base de adicionales fuera de esos place ids
# concatenar ambos resultados

In [ ]:
PRED_CAT_FINAL_N.rename(columns={'CALLEINTFIS_RS_FD_': 'INTFIS_RS_FD_CALLEINTFIS_RS_FD_', 'COLONIAINTFIS_RS_FD_': 'INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'},inplace=True)

In [ ]:
PRED_CAT_FINAL_N.to_csv('BASE_PREDIAL_CATASTRO_ECATEPEC.csv', encoding='utf-8-sig', index=False)

# Para CURTS y el otro query

In [ ]:
CURT_FA=PRED_CAT_FINAL_N.loc[(PRED_CAT_FINAL_N['INTFIS_RS_FD_DIRECCION'].isna()) & (PRED_CAT_FINAL_N['CURT'].notna())]
CURT_FA=CURT_FA[['CLAVECATASTRAL', 'DIRECCION_FINAL',
       'ID_MUNICIPIO', 'ORDEN', 'CLAVECATASTRAL_RECAUDACION',
       'BDINTERNA_CONT_NOMBRE_COMPLETO', 'BDINTERNA_CONT_RFC',
       'BDINTERNA_CONT_EMAIL', 'BDINTERNA_CONT_DIRECCION',
       'BD_INTERNA_FUENTE_ORIGEN', 'CURT', 'ESTATUS_2007', 'ESTATUS_2008',
       'ESTATUS_2009', 'ESTATUS_2010', 'ESTATUS_2011', 'ESTATUS_2012',
       'ESTATUS_2013', 'ESTATUS_2014', 'ESTATUS_2015', 'ESTATUS_2016',
       'ESTATUS_2017', 'ESTATUS_2018', 'ESTATUS_2019', 'ESTATUS_2020',
       'ESTATUS_2021', 'ESTATUS_2022', 'VALORCATASTRAL_2007',
       'VALORCATASTRAL_2008', 'VALORCATASTRAL_2009', 'VALORCATASTRAL_2010',
       'VALORCATASTRAL_2011', 'VALORCATASTRAL_2012', 'VALORCATASTRAL_2013',
       'VALORCATASTRAL_2014', 'VALORCATASTRAL_2015', 'VALORCATASTRAL_2016',
       'VALORCATASTRAL_2017', 'VALORCATASTRAL_2018', 'VALORCATASTRAL_2019',
       'VALORCATASTRAL_2020', 'VALORCATASTRAL_2021', 'VALORCATASTRAL_2022',
       'USO', 'PREDIAL_PROPIETARIO_NOMBRE', 'PREDIAL_PROPIETARIO_CALLE',
       'PROPIETARIO_ASENTAMIENTO', 'PROPIETARIO_CP', 'CATASTRO_MUNICIPIO',
       'CATASTRO_ZONA', 'CATASTRO_MANZANA', 'CATASTRO_LOTE',
       'CATASTRO_EDIFICIO', 'CATASTRO_DEPARTAMENTO',
       'CATASTRO_ANIO_CONSTRUCCION', 'CATASTRO_ESTADO_CONSTRUCCION',
       'CATASTRO_NIVELES_CONSTRUCCION', 'CATASTRO_DOMICILIO_INMUEBLE',
       'CATASTRO_DOMICILIO_INMUEBLE_CONSTRUIDO',
       'CATASTRO_CANTIDAD_REPITE_CVECATASTRAL', 'NOMBRE_MUNICIPIO',
       'FECHA_CORTE', 'PAGO_AL_MENOS_UNO', 'CATASTRO_DOMICILIO_CATASTRO']]

In [ ]:
pcf_n1=PRED_CAT_FINAL_N.loc[~(PRED_CAT_FINAL_N['ORDEN'].isin(CURT_FA['ORDEN']))]

In [ ]:
CURT_FA_0= pd.read_csv('Downloads/ECATEPEC_PARA_REV_GEO_procesado.txt', sep='\t', encoding='utf-8-sig')

In [ ]:
CURT_FA_0.rename(columns={'DIRECCION':'INTFIS_RS_FD_DIRECCION', 'place_id':'INTFIS_RS_FD_PLACE_ID', 'types':'INTFIS_RS_FD_TYPES',
        'geometry.location.lat':'INTFIS_RS_FD_LAT', 'geometry.location.lng':'INTFIS_RS_FD_LON', 'geometry.location_type':'INTFIS_RS_FD_GEOM_TYPE',
       'plus_code.compound_code':'INTFIS_RS_FD_PLUS_CODE'}, inplace=True)

In [ ]:
def load_data_pols(GL_P1):
    # GL_P1 = pd.read_csv(self.city_file_pols, sep=",", encoding="utf-8-sig",)
    GL_P1.rename(columns={'DIRECCION':'DIRECCION_CATS'}, inplace=True)
    GL_P1["GEO_GEOMETRY_1"] = GL_P1["GEO_GEOMETRY"].str.replace(
        "POINT", "", regex=False
    )
    GL_P1["GEO_GEOMETRY_1"] = GL_P1["GEO_GEOMETRY_1"].str.replace(
        ")", "", regex=False
    )
    GL_P1["GEO_GEOMETRY_1"] = GL_P1["GEO_GEOMETRY_1"].str.replace(
        "(", "", regex=False
    )
    GL_P1["GEO_GEOMETRY_1"] = GL_P1["GEO_GEOMETRY_1"].str.replace(
        ",", "", regex=False
    )
    n = (
        GL_P1["GEO_GEOMETRY_1"]
        .str.split(" ", expand=True)
        .rename(columns={1: "lon", 2: "lat"})
    )
    GL_S = pd.concat([GL_P1, n], axis=1)

    # Redondeamos a 4 decimales, que es una precision de ~10 m para
    # reducir el numero de busquedas que tenemos que hacer
    GL_S["lon"] = GL_S["lon"].astype(float)
    GL_S["lat"] = GL_S["lat"].astype(float)
    GL_S["latlong"] = GL_S["lat"].astype(str) + ", " + GL_S["lon"].astype(str)

    return(GL_S)

In [ ]:

GL_S=load_data_pols(pd.merge(CURT_FA, pred[['id_cat','GEO_GEOMETRY']], left_on='CLAVECATASTRAL', right_on='id_cat'))

In [ ]:
# CURT_FA_0=pd.merge(GL_S,CURT_FA_0 , on='latlong').rename(columns={'DIRECCION':'INTFIS_RS_FD_DIRECCION', 'lon':'INTFIS_RS_FD_LON', 'lat':'INTFIS_RS_FD_LAT'})

In [ ]:
PRED_CAT_FINAL_N2=pd.concat([pcf_n1, CURT_FA_0], axis=0)

In [ ]:
PRED_CAT_FINAL_N3.loc[~PRED_CAT_FINAL_N3['CLAVECATASTRAL_RECAUDACION'].isin(PRED_CAT_FINAL_N['CLAVECATASTRAL_RECAUDACION'])]
PRED_CAT_FINAL_N.loc[~PRED_CAT_FINAL_N['CLAVECATASTRAL_RECAUDACION'].isin(PRED_CAT_FINAL_N3['CLAVECATASTRAL_RECAUDACION'])][['CLAVECATASTRAL', 'DIRECCION_FINAL',
       'ID_MUNICIPIO', 'ORDEN', 'CLAVECATASTRAL_RECAUDACION',
       'BDINTERNA_CONT_NOMBRE_COMPLETO', 'BDINTERNA_CONT_RFC',
       'BDINTERNA_CONT_EMAIL', 'BDINTERNA_CONT_DIRECCION',
       'BD_INTERNA_FUENTE_ORIGEN', 'CURT', 'ESTATUS_2007', 'ESTATUS_2008',
       'ESTATUS_2009', 'ESTATUS_2010', 'ESTATUS_2011', 'ESTATUS_2012',
       'ESTATUS_2013', 'ESTATUS_2014', 'ESTATUS_2015', 'ESTATUS_2016',
       'ESTATUS_2017', 'ESTATUS_2018', 'ESTATUS_2019', 'ESTATUS_2020',
       'ESTATUS_2021', 'ESTATUS_2022', 'VALORCATASTRAL_2007',
       'VALORCATASTRAL_2008', 'VALORCATASTRAL_2009', 'VALORCATASTRAL_2010',
       'VALORCATASTRAL_2011', 'VALORCATASTRAL_2012', 'VALORCATASTRAL_2013',
       'VALORCATASTRAL_2014', 'VALORCATASTRAL_2015', 'VALORCATASTRAL_2016',
       'VALORCATASTRAL_2017', 'VALORCATASTRAL_2018', 'VALORCATASTRAL_2019',
       'VALORCATASTRAL_2020', 'VALORCATASTRAL_2021', 'VALORCATASTRAL_2022',
       'USO', 'PREDIAL_PROPIETARIO_NOMBRE', 'PREDIAL_PROPIETARIO_CALLE',
       'PROPIETARIO_ASENTAMIENTO', 'PROPIETARIO_CP', 'CATASTRO_MUNICIPIO',
       'CATASTRO_ZONA', 'CATASTRO_MANZANA', 'CATASTRO_LOTE',
       'CATASTRO_EDIFICIO', 'CATASTRO_DEPARTAMENTO',
       'CATASTRO_ANIO_CONSTRUCCION', 'CATASTRO_ESTADO_CONSTRUCCION',
       'CATASTRO_NIVELES_CONSTRUCCION', 'CATASTRO_DOMICILIO_INMUEBLE',
       'CATASTRO_DOMICILIO_INMUEBLE_CONSTRUIDO',
       'CATASTRO_CANTIDAD_REPITE_CVECATASTRAL', 'NOMBRE_MUNICIPIO',
       'FECHA_CORTE', 'PAGO_AL_MENOS_UNO']].to_csv('SIN_GEO_ECATEPEC_CAT_PREDIAL.csv', encoding='utf-8-sig')

In [ ]:
PRED_CAT_FINAL_N4=PRED_CAT_FINAL_N3.loc[PRED_CAT_FINAL_N3['USO']=='H']

In [ ]:
# PRED_CAT_FINAL_N2=PRED_CAT_FINAL_N2.drop(columns=0)

In [ ]:
PRED_CAT_FINAL_N2=PRED_CAT_FINAL_N2[['CLAVECATASTRAL', 'DIRECCION_FINAL',
       'ID_MUNICIPIO', 'ORDEN', 'CLAVECATASTRAL_RECAUDACION',
       'BDINTERNA_CONT_NOMBRE_COMPLETO', 'BDINTERNA_CONT_RFC',
       'BDINTERNA_CONT_EMAIL', 'BDINTERNA_CONT_DIRECCION',
       'BD_INTERNA_FUENTE_ORIGEN', 'CURT', 'ESTATUS_2007', 'ESTATUS_2008',
       'ESTATUS_2009', 'ESTATUS_2010', 'ESTATUS_2011', 'ESTATUS_2012',
       'ESTATUS_2013', 'ESTATUS_2014', 'ESTATUS_2015', 'ESTATUS_2016',
       'ESTATUS_2017', 'ESTATUS_2018', 'ESTATUS_2019', 'ESTATUS_2020',
       'ESTATUS_2021', 'ESTATUS_2022', 'VALORCATASTRAL_2007',
       'VALORCATASTRAL_2008', 'VALORCATASTRAL_2009', 'VALORCATASTRAL_2010',
       'VALORCATASTRAL_2011', 'VALORCATASTRAL_2012', 'VALORCATASTRAL_2013',
       'VALORCATASTRAL_2014', 'VALORCATASTRAL_2015', 'VALORCATASTRAL_2016',
       'VALORCATASTRAL_2017', 'VALORCATASTRAL_2018', 'VALORCATASTRAL_2019',
       'VALORCATASTRAL_2020', 'VALORCATASTRAL_2021', 'VALORCATASTRAL_2022',
       'USO', 'PREDIAL_PROPIETARIO_NOMBRE', 'PREDIAL_PROPIETARIO_CALLE',
       'PROPIETARIO_ASENTAMIENTO', 'PROPIETARIO_CP', 'CATASTRO_MUNICIPIO',
       'CATASTRO_ZONA', 'CATASTRO_MANZANA', 'CATASTRO_LOTE',
       'CATASTRO_EDIFICIO', 'CATASTRO_DEPARTAMENTO',
       'CATASTRO_ANIO_CONSTRUCCION', 'CATASTRO_ESTADO_CONSTRUCCION',
       'CATASTRO_NIVELES_CONSTRUCCION', 'CATASTRO_DOMICILIO_INMUEBLE',
       'CATASTRO_DOMICILIO_INMUEBLE_CONSTRUIDO',
       'CATASTRO_CANTIDAD_REPITE_CVECATASTRAL', 'NOMBRE_MUNICIPIO',
       'FECHA_CORTE', 'PAGO_AL_MENOS_UNO', 'INTFIS_RS_FD_PARTIAL_MATCH',
       'INTFIS_RS_FD_KEY', 'CATASTRO_DOMICILIO_CATASTRO',
       'INTFIS_RS_FD_PRE_DOM', 'INTFIS_RS_FD_N_EXT',
       'INTFIS_RS_FD_PRE_N_EXT', 'INTFIS_RS_FD_CALLE', 'INTFIS_RS_FD_COLONIA',
       'INTFIS_RS_FD_CALLE_NORM', 'INTFIS_RS_FD_COLONIA_NORM', 'CALLE',
       'INTFIS_RS_FD_CALLEINTFIS_RS_FD_', 'COLONIA',
       'INTFIS_RS_FD_COLONIAINTFIS_RS_FD_', 'INTFIS_RS_FD_FULL_DOM','INTFIS_RS_FD_DIRECCION', 'INTFIS_RS_FD_PLACE_ID', 'INTFIS_RS_FD_TYPES',
       'INTFIS_RS_FD_LAT', 'INTFIS_RS_FD_LON', 'INTFIS_RS_FD_GEOM_TYPE',
       'INTFIS_RS_FD_PLUS_CODE']]

In [ ]:
PRED_CAT_FINAL_N2=creacion_urls(PRED_CAT_FINAL_N2)

In [ ]:
PRED_CAT_FINAL_N3=PRED_CAT_FINAL_N2.loc[((PRED_CAT_FINAL_N2['INTFIS_RS_FD_DIRECCION'].notna())& (~PRED_CAT_FINAL_N2['INTFIS_RS_FD_TYPES'].astype(str).str.contains('locality|establishment|sublocality')))]

In [ ]:
PRED_CAT_FINAL_N3.to_csv('BASE_PREDIAL_CATASTRO_ECATEPEC.csv', encoding='utf-8-sig')

In [ ]:
CURT_FA.loc[CURT_FA['CLAVECATASTRAL'].isin(pred['id_cat'])]
pd.merge(CURT_FA, pred[['id_cat','GEO_GEOMETRY']], left_on='CLAVECATASTRAL', right_on='id_cat').to_csv('CURTS_REVS_EXTRAS.csv', encoding='utf-8-sig')

### CREAR FUNCION A PARTIR DE :

In [ ]:
AG1=ads_reps.groupby(['latlon']).count().sort_values('INTFIS_RS_FD_PLACE_ID', ascending=False).reset_index()[['latlon', 'INTFIS_RS_FD_PLACE_ID']].rename(columns={'INTFIS_RS_FD_PLACE_ID':'conteo_coords'})
MAG=pd.merge(ads_reps, AG1, on='latlon')
MAG['INTFIS_RS_FD_TYPES']=MAG['INTFIS_RS_FD_TYPES'].str.replace("'",'').str.replace("s,t,r,e,e,t,_,a,d,d,r,e,s,s",'street_address').str.replace("s, t, r, e, e, t, _, a, d, d, r, e, s, s",'street_address').str.replace("p, r, e, m, i, s, e",'premise').str.replace("r, o, u, t, e",'route')
MAG.loc[MAG['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'].fillna('NaN').str.fullmatch(r"(\d{1}\-?\d{0,4})"),'INTFIS_RS_FD_CP']=MAG.loc[MAG['INTFIS_RS_FD_CP'].fillna('NaN').str.fullmatch(r"(\d{1}\-?\d{0,4})")]['INTFIS_RS_FD_DIRECCION'].str.split(r"(\d{1}\-?\d{0,4})").str[-4]
MAG.loc[MAG['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'].fillna('NaN').str.fullmatch(r"(\d{1}\-?\d{0,4})"),'INTFIS_RS_FD_CALLE_NORM']=MAG.loc[MAG['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'].fillna('NaN').str.fullmatch(r"(\d{1}\-?\d{0,4})")]['INTFIS_RS_FD_DIRECCION'].str.split(r"(\d{1}\-?\d{0,4})|S/N").str[-5].str.replace(',','').str.lower()
MAG.loc[MAG['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'].fillna('NaN').str.fullmatch(r"(\d{1}\-?\d{0,4})"),'INTFIS_RS_FD_COLONIAINTFIS_RS_FD_']=MAG.loc[MAG['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'].fillna('NaN').str.fullmatch(r"(\d{1}\-?\d{0,4})")]['INTFIS_RS_FD_DIRECCION'].str.split(r"(\d{1}\-?\d{0,4})|S/N").str[-3].str.replace(',','').str.lower()

MAG.loc[MAG['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'].fillna('NaN').str[-5:].str.contains(r"(\d{1}\-?\d{0,4})"),'INTFIS_RS_FD_CP']=MAG.loc[MAG['INTFIS_RS_FD_CP'].fillna('NaN').str[-5:].str.contains(r"(\d{1}\-?\d{0,4})")]['INTFIS_RS_FD_DIRECCION'].str.split(r"(\d{1}\-?\d{0,4})").str[-2].str.replace(',','').str.lower()
MAG.loc[MAG['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'].fillna('NaN').str[-5:].str.contains(r"(\d{1}\-?\d{0,4})"),'INTFIS_RS_FD_CALLE_NORM']=MAG.loc[MAG['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'].fillna('NaN').str[-5:].str.contains(r"(\d{1}\-?\d{0,4})")]['INTFIS_RS_FD_DIRECCION'].str.split(r"(\d{1}\-?\d{0,4})|S/N").str[0].str.replace(',','').str.lower()
MAG.loc[MAG['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'].fillna('NaN').str[-5:].fillna('NaN').str[-5:].str.contains(r"(\d{1}\-?\d{0,4})"),'INTFIS_RS_FD_COLONIAINTFIS_RS_FD_']=MAG.loc[MAG['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'].fillna('NaN').str[-5:].str.contains(r"(\d{1}\-?\d{0,4})")]['INTFIS_RS_FD_DIRECCION'].str.split(r",").str[-4].str.strip().str.lower()
AG2 =MAG.groupby(['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_','INTFIS_RS_FD_CALLE_NORM']).count().sort_values('INTFIS_RS_FD_PLACE_ID', ascending=False).reset_index()[['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_','INTFIS_RS_FD_CALLE_NORM', 'INTFIS_RS_FD_PLACE_ID']].rename(columns={'INTFIS_RS_FD_PLACE_ID':'conteo_doms'})
MAG2=pd.merge(MAG, AG2, on=['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_','INTFIS_RS_FD_CALLE_NORM'])
cols=MAG.groupby(['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_']).count().sort_values('INTFIS_RS_FD_PLACE_ID', ascending=False).reset_index()[['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_','INTFIS_RS_FD_PLACE_ID']].rename(columns={'INTFIS_RS_FD_PLACE_ID':'conteo_cols'}).sort_values('INTFIS_RS_FD_COLONIAINTFIS_RS_FD_')
MAG3=pd.merge(MAG2, cols, on=['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'])

joined_MAG3=group_join(MAG3)
# MAG.groupby(['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_','INTFIS_RS_FD_CALLEINTFIS_RS_FD_']).count()

CASAS=joined_MAG3.loc[joined_MAG3['join_INTFIS_RS_FD_TYPES'].str.contains('street_address|premise|route')]
ESTB=joined_MAG3.loc[~joined_MAG3['join_INTFIS_RS_FD_TYPES'].str.contains('street_address|premise|route')]

PRE_f=pd.merge(CASAS,MAG3[['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_','INTFIS_RS_FD_CALLEINTFIS_RS_FD_', 'latlon']], on=['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_','INTFIS_RS_FD_CALLEINTFIS_RS_FD_']).drop_duplicates()

P=MAG3.groupby(['latlon']).mean()
FINAL_GROUP=pd.merge(P[['conteo_cols']].reset_index(), PRE_f, on='latlon').drop_duplicates()

PP=MAG3.groupby(['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_','INTFIS_RS_FD_CALLEINTFIS_RS_FD_']).max()
FINAL_GROUP_1=pd.merge(PP[['conteo_coords']].reset_index(), FINAL_GROUP, on=['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_','INTFIS_RS_FD_CALLEINTFIS_RS_FD_']).drop(columns='latlon').drop_duplicates()[['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_', 'INTFIS_RS_FD_CALLEINTFIS_RS_FD_',
       'conteo_coords', 'conteo_cols', 'conteo_doms','INTFIS_RS_FD_LAT','INTFIS_RS_FD_LON',
       'join_INTFIS_RS_FD_TYPES',
       'join_INTFIS_RS_FD_PLUS_CODE', 'join_INTFIS_RS_FD_N_EXT',
       'join_INTFIS_RS_FD_CP']]

ESTB=ads_reps[(~ads_reps['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'].isin(FINAL_GROUP_1['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'])) & (~ads_reps['INTFIS_RS_FD_CALLEINTFIS_RS_FD_'].isin(FINAL_GROUP_1['INTFIS_RS_FD_CALLEINTFIS_RS_FD_']))]
ESTB.loc[~ESTB['INTFIS_RS_FD_TYPES'].str.contains('establishment')]
MAG.loc[MAG['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'].astype(str).str.contains('novela')]
N=MAG.groupby(['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_','INTFIS_RS_FD_CALLE_NORM']).count().reset_index()
N.loc[N['INTFIS_RS_FD_COLONIAINTFIS_RS_FD_'].astype(str).str.contains('mexicana')]


FINAL_GROUP_1=creacion_urls(FINAL_GROUP_1)